# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments)
- ArticlesApril2018.csv

In [1]:
import numpy as np
import pandas as pd
from string import punctuation
from google.colab import files
file_uploaded = files.upload()

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [2]:
seed = 2021
np.random.seed(seed)

## 1.데이터 전처리

In [3]:
filename = list(file_uploaded.keys())[0]
df=pd.read_csv(filename, encoding='latin1')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Torontoâs ...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
df.headline.isnull().values.any()

False

In [6]:
headline = [text for text in df.headline.values if text != 'Unknown']
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [7]:
len(headline)

1214

In [8]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [9]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print(f'단어집합의 크기 : {vocab_size}')

단어집합의 크기 : 3494


In [11]:
sequences = []
for line in text:   # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:5]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52]]

In [12]:
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key

print(f'빈도수 상위 1번단어 : {index_to_word[1]}')
print(f'빈도수 상위 582번단어 : {index_to_word[582]}')

빈도수 상위 1번단어 : the
빈도수 상위 582번단어 : offer


In [13]:
max_len = max(len(s) for s in sequences)
print(f'샘플 최대 길이 : {max_len}')

샘플 최대 길이 : 24


In [14]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [15]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [16]:
y = to_categorical(y, num_classes=vocab_size)

In [17]:
X.shape, y.shape

((7803, 23), (7803, 3494))

## 2.모델 설계 및 학습

In [18]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [19]:
model = Sequential([
                    Embedding(vocab_size, 10, input_length=max_len-1,
                              name="Embedding_Layer"),
                    LSTM(128,name="LSTM_Layer"),
                    Dense(vocab_size, activation="softmax",name="Output_Layer")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [21]:
history = model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 9s - loss: 7.6414 - accuracy: 0.0293
Epoch 2/200
244/244 - 1s - loss: 7.1089 - accuracy: 0.0315
Epoch 3/200
244/244 - 1s - loss: 6.9695 - accuracy: 0.0343
Epoch 4/200
244/244 - 1s - loss: 6.8330 - accuracy: 0.0417
Epoch 5/200
244/244 - 1s - loss: 6.6767 - accuracy: 0.0456
Epoch 6/200
244/244 - 1s - loss: 6.5013 - accuracy: 0.0492
Epoch 7/200
244/244 - 1s - loss: 6.3081 - accuracy: 0.0557
Epoch 8/200
244/244 - 1s - loss: 6.1076 - accuracy: 0.0598
Epoch 9/200
244/244 - 1s - loss: 5.9155 - accuracy: 0.0637
Epoch 10/200
244/244 - 1s - loss: 5.7329 - accuracy: 0.0684
Epoch 11/200
244/244 - 1s - loss: 5.5629 - accuracy: 0.0757
Epoch 12/200
244/244 - 1s - loss: 5.4035 - accuracy: 0.0755
Epoch 13/200
244/244 - 1s - loss: 5.2476 - accuracy: 0.0836
Epoch 14/200
244/244 - 1s - loss: 5.1015 - accuracy: 0.0887
Epoch 15/200
244/244 - 1s - loss: 4.9623 - accuracy: 0.0994
Epoch 16/200
244/244 - 1s - loss: 4.8305 - accuracy: 0.1078
Epoch 17/200
244/244 - 1s - loss: 4.6991 - accura

## 3. 모델 검증

In [24]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [25]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i disapprove of school vouchers can i still apply for them


In [26]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how do you feel about being told to smile paper boi


In [27]:
print(sentence_generation(model, t, 'former', 10))

former judge to review files seized in searches fought recalls it
